# Yield calculator

This notebook is used to estimate the CCSNe yields consistent with the present-day APOGEE trend in \[C/Mg\]-\[Mg/H\].

uses the results from the MCMC fits to AGB integrated yields with 

In [ ]:
import toml
import surp
import vice

import vice.yields.ccsne.settings as y_cc
import vice.yields.agb.settings as Y_agb
import vice.yields.sneia.settings as y_ia

In [ ]:
surp.set_yields(verbose=False)

# yield params calculator

Given the desired f_agb or alpha, what are the required y0cc and alpha_agb values

## inputs from mcmc fits

In [ ]:
Y_C_AGB = {
    "cristallo11": 3.8e-4,
    "ventura13": 1.4e-4,
    "karakas16": 2.8e-4,
    "pignatari16": 8.3e-4
}

In [ ]:
for label, yagb in Y_C_AGB.items():
    print(label, "\t", yagb / Y_C_0)

In [ ]:
ZETA_C_AGB = {
    "cristallo11": -3.5e-4,
    "ventura13": -9.4e-4,
    "karakas16": -10.1e-4,
    "pignatari16": -5.7e-4
}

In [ ]:
# tuned to fruity
y_mg = y_cc["mg"]
Y_C_0 = 2.75e-3 # pm 0.16
ZETA_C_0 = 1.52e-3 # pm 0.32 (for fruity)
A_C_0 = 2.73 # pm 0.5

## Utils

In [ ]:
def f_agb_to_alpha(y0_agb, f_agb):
    y_agb = Y_C_0 * f_agb
    return y_agb / y0_agb

In [ ]:
def f_agb_to_settings(study, y0=None, f_agb=0.2):
    if y0 is None:
        y0 = Y_C_AGB[study]
        
    alpha = f_agb_to_alpha(y0, f_agb)
    y0_cc = (1-f_agb) * Y_C_0
    settings = dict(alpha_c_agb = alpha, y0_c_cc = y0_cc)
    
    print(toml.dumps(settings))

In [ ]:
def f_agb_zeta_to_settings(study, *, y0_agb=None, zeta_agb=None, f_agb=0.2):
    if y0_agb is None:
        y0_agb = Y_C_AGB[study]
    if zeta_agb is None:
        zeta_agb = ZETA_C_AGB[study]
        
    alpha = f_agb_to_alpha(y0_agb, f_agb)
    y0_cc = (1-f_agb) * Y_C_0
    
    zeta_cc = ZETA_C_0 - zeta_agb * alpha
    
    settings = dict(alpha_c_agb = alpha, y0_c_cc = y0_cc, zeta_c_cc=zeta_cc)
    
    print(toml.dumps(settings))

In [ ]:
def alpha_to_settings(study, alpha):
    f_agb =  alpha * Y_C_AGB[study] / Y_C_0
    y0_cc = (1-f_agb) * Y_C_0
    settings = dict(alpha_c_agb = alpha, y0_c_cc = y0_cc)
    
    print(toml.dumps(settings))

In [ ]:
def enhance_fe_ia(factor):
    """if factor != 1, then enhances sne ia fe yield by factor but maintains same total fe yield, applied to params"""
    if factor == 1:
        return
    y_fe = y_ia["fe"] + y_cc["fe"]
    y_fe_ia = y_ia["fe"] * factor
    y_fe_cc = y_fe - y_fe_ia
    
    print(f"y_fe_ia = {y_fe_ia:0.8f}")
    print(f"y_fe_cc = {y_fe_cc:0.8f}")
    return y_fe_ia, y_fe_cc


In [ ]:
def m_factor_to_settings(Y_c_agb="cristallo11"):
    surp.set_yields(y_c_cc=0, Y_c_agb=Y_c_agb, kwargs_c_agb = dict(mass_factor=m), verbose=False)
    y_agb = surp.yields.calc_y(surp.Z_SUN)
    
    print(y_cc_0 - surp.yields.calc_y(surp.Z_SUN) + y_agb_0)

## Fruity

In [ ]:
alpha_to_settings("cristallo11", alpha=1.449)

In [ ]:
enhance_fe_ia(1.2)

In [ ]:
for f in [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.8, 1]:
    print("f_agb = ", f)
    f_agb_zeta_to_settings("cristallo11", f_agb=f)
    print()

## ATON

In [ ]:
f_agb_zeta_to_settings("ventura13", f_agb=0.2)


In [ ]:
alpha_to_settings("ventura13", alpha=2)
print()
alpha_to_settings("ventura13", alpha=3)

## Monash

In [ ]:
f_agb_zeta_to_settings("karakas16", f_agb=0.2)


In [ ]:
for f in [0, 0.1, 0.3, 0.4, 0.5, 1]:
    print(f)
    f_agb_zeta_to_settings("karakas16", f_agb=f)

## NuGrid

In [ ]:
f_agb_zeta_to_settings("pignatari16", f_agb=0.2)

In [ ]:
for f in [0, 0.1, 0.3, 0.5, 1]:
    f_agb_zeta_to_settings("pignatari16", f_agb=f)

### Mass factor

In [ ]:
for m in [0.5, 0.8, 1, 1.5, 2]:
    surp.set_yields(y_c_cc=0, kwargs_c_agb = dict(mass_factor=m), verbose=False)
    print(m)
    print(surp.yields.calc_y(surp.Z_SUN))
    print(y_cc_0 - surp.yields.calc_y(surp.Z_SUN) + y_agb_0)

In [ ]:
y_agb_0 = 0.00046798503839536314
y_cc_0 = 0.0022027

In [ ]:
for m in [0.5, 0.8, 1, 1.5, 2]:
   

## Analytic

In [ ]:
y0_agb_a = Y_C_AGB["cristallo11"]
y0_agb_a

In [ ]:
zeta_agb_a = ZETA_C_AGB["cristallo11"]
zeta_agb_a

In [ ]:
def get_alpha(f_agb=0.2, y0_agb=y0_agb_a):
    return Y_C_0 / y0_agb * f_agb

In [ ]:
def get_zeta_cc(alpha, zeta=zeta_agb_a):
    return ZETA_C_0 - alpha*zeta

In [ ]:
def get_ana_settings(f, zeta=zeta_agb_a):
    alpha = get_alpha(f)
    print("alpha_c_agb = ", alpha)
    print("y0_c_cc = ", Y_C_0 - alpha * y0_agb_a)
    print("zeta_c_cc = ", get_zeta_cc(alpha, zeta))

In [ ]:
get_ana_settings(0.2)

In [ ]:
get_ana_settings(0.2, 0)

In [ ]:
get_ana_settings(0.2, -0.0007)

In [ ]:
for f in [0, 0.1, 0.3, 0.5, 1]:
    print(f)
    get_ana_settings(f)
    print()

In [ ]:
y = surp.yield_models.C_AGB_Model()

In [ ]:
print(y)

In [ ]:
print(2*y)

In [ ]:
print(0.15 * y)